# **NN BASELINE**
In this notebook i have implemented a baseline model of Neural Network (do let me know if you find any doubts or bugs)

In [ ]:


from IPython.core.display import display, HTML

import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import gc

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib


path_submissions = '/'

target_name = 'target'
scores_folds = {}

import os      
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/train.csv")
test=pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/test.csv")

In [ ]:
print("train shape is:",train.shape)
print("test shape is:",test.shape)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# train.isnull().sum()

In [ ]:
train['target'].unique()

# OBSERVATIONS
1. **No null value is present in the dataset**
2. **data has 102 columns/features including target**
3. **100 features aree float 2 are integers**
4. **The given problem is of binary classification.**



In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
from numpy.random import seed
seed(42)
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow import keras
import numpy as np
from keras import backend as K
from sklearn.metrics import accuracy_score
def root_mean_squared_per_error(y_true, y_pred):
         return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true )))
    
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, verbose=0,
    mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=7, verbose=0,
    mode='min')

In [ ]:
colNames=[col for col in test.columns if col not in 'id']
# colNames.remove('id')
train.replace([np.inf, -np.inf], np.nan,inplace=True)
test.replace([np.inf, -np.inf], np.nan,inplace=True)
qt_train = []
train_nn=train[colNames].copy()
test_nn=test[colNames].copy()
# for col in colNames:
#     #print(col)
#     qt = QuantileTransformer(random_state=21,n_quantiles=2000, output_distribution='normal')
#     train_nn[col] = qt.fit_transform(train_nn[[col]])
#     test_nn[col] = qt.transform(test_nn[[col]])    
#     qt_train.append(qt)


In [ ]:
train_nn['target']=train['target']

In [ ]:
train_nn

In [ ]:
#https://bignerdranch.com/blog/implementing-swish-activation-function-in-keras/
from keras.backend import sigmoid
from sklearn.metrics import roc_auc_score
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})

hidden_units = (100,64,32)#initialized number of neurons in each hidden layer

def base_model():

    num_input = keras.Input(shape=(100,), name='num_data')#input layer


    out = keras.layers.Concatenate()([num_input])
    
    # Add one or more hidden layers
    for n_hidden in hidden_units:

        out = keras.layers.Dense(n_hidden, activation='swish')(out)
        

    #out = keras.layers.Concatenate()([out, num_input])

    # A single output: our predicted target value probability
    out = keras.layers.Dense(1, activation='sigmoid', name='prediction')(out)
    
    model = keras.Model(
    inputs = [num_input],
    outputs = out,
    )
    
    return model

In [ ]:
target_name='target'
scores_folds = {}
model_name = 'NN'
pred_name = 'pred_{}'.format(model_name)

n_folds = 5
kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=12)
scores_folds[model_name] = []
counter = 1

features_to_consider = list(train_nn)

features_to_consider.remove('target')
try:
    features_to_consider.remove('pred_NN')
except:
    pass


train_nn[features_to_consider] = train_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
test_nn[features_to_consider] = test_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())

train_nn[pred_name] = 0
test_nn[target_name] = 0
test_predictions_nn = np.zeros(test_nn.shape[0])

for fold, (trn_ind, val_ind) in enumerate(kf.split(train)):
    print(f'Training fold {fold + 1}')
    X_train, X_test = train_nn.iloc[trn_ind][features_to_consider], train_nn.iloc[val_ind][features_to_consider]
    y_train, y_test = train_nn.iloc[trn_ind]['target'], train_nn.iloc[val_ind]['target']
    print('CV {}/{}'.format(counter, n_folds)) 
    #############################################################################################
    # NN
    #############################################################################################
    
    model = base_model()
    
    model.compile(
        keras.optimizers.Adam(learning_rate=0.0007),
        loss='binary_crossentropy',
        metrics = ['AUC']
    )
    
    num_data = X_train[features_to_consider]
    
    scaler = MinMaxScaler(feature_range=(0, 1))         
    num_data = scaler.fit_transform(num_data.values)    
      
    target =  y_train
    
    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)

    model.fit([num_data], 
              target,              
              batch_size=2048,
              epochs=700,
              validation_data=([num_data_test], y_test),
              callbacks=[es, plateau],
              validation_batch_size=len(y_test),
              shuffle=True,
             verbose = 1)

    preds = model.predict([num_data_test]).reshape(1,-1)[0]
    score = round(roc_auc_score(y_test,preds),5)
    print('Fold {} {}: {}'.format(counter, model_name, score))
    scores_folds[model_name].append(score)
    
    tt =scaler.transform(test_nn[features_to_consider].values)
    
    test_predictions_nn += model.predict([tt]).reshape(1,-1)[0].clip(0,1e10)/n_folds       
    counter += 1

In [ ]:
sub=pd.read_csv("../input/tabular-playground-series-nov-2021/sample_submission.csv")
sub

In [ ]:
sub['target']=test_predictions_nn

In [ ]:
sub

In [ ]:
sub.to_csv("submission.csv",index=False)